# Imports 

In [ ]:
from PIL import Image
import numpy as np
from skimage import measure
import time
import gc
from matplotlib import pyplot as plt
import openslide as op

# Configuration

In [ ]:
mask = Image.open("/root/workspace/src/prediction_15-02_voley.png") # file with the predictions
filename = '/root/workspace/data/SVS_test/IFTA_12_02.svs' # original image
transparency = 110  # alpha channel on the output image
output_image='/root/workspace/data/res.png' # name of the output image
display = True  # if true, display the images during execution

In [ ]:
FN = 0 # Number of false negatives
TP = 0 # Number of true positives
FP = 0 # Number of false positives

# Code Execution

In [ ]:
# Get the data
mask = mask.convert("RGBA")
maskArray = np.array(mask)

In [ ]:
# Changing the channels (R and B are normally prediction and G ground truth)
maskArray[:,:,2]=maskArray[:,:,1]

In [ ]:
# Creating pointing variable for readibility 
pred = maskArray[:,:,0]
gt = maskArray[:,:,2]

In [ ]:
# Finding the connected components
maskArray[:,:,0] = measure.label(pred)
pred_nb = pred.max()
maskArray[:,:,2] = measure.label(gt)
gt_nb = gt.max()

In [ ]:
# Initializing arrays with the connected parts coordinates
bb_pred = np.zeros((pred_nb,4))
bb_gt = np.zeros((gt_nb,4))

In [ ]:
# Filling the arrays with coordinate
for i in range(1,pred_nb+1):
    indices = np.where(pred==i)
    xmin = bb_pred[i-1][0] = indices[0].min()
    xmax = bb_pred[i-1][1] = indices[0].max()
    ymin = bb_pred[i-1][2] = indices[1].min()
    ymax = bb_pred[i-1][3] = indices[1].max()
    pred[xmin:xmax+1,ymin:ymax+1]=i

for i in range(1,gt.max()+1):
    indices = np.where(gt==i)
    xmin = bb_gt[i-1][0] = indices[0].min()
    xmax = bb_gt[i-1][1] = indices[0].max()
    ymin = bb_gt[i-1][2] = indices[1].min()
    ymax = bb_gt[i-1][3] = indices[1].max()
    gt[xmin:xmax+1,ymin:ymax+1]=i

In [ ]:
# these 2 arrays are a bit tricky, the idea is to find the corresponding parts of an intersection. 
# By adding the number of parts, we are sure to have only one component.
# The first array is to find the prediction component and the second one the ground truth component 
all_pred = np.copy(gt)
np.putmask(all_pred,all_pred!=0,pred_nb)
all_pred += pred

all_gt = np.copy(pred)
np.putmask(all_gt,all_gt!=0,gt_nb)
all_gt += gt

In [ ]:
# Display the result
if(display):
    plt.figure(figsize=(50,50))
    plt.subplot(131)
    plt.imshow(pred, cmap='nipy_spectral')
    plt.subplot(132)
    plt.imshow(gt, cmap='nipy_spectral')
    plt.subplot(133)
    plt.imshow(all_pred, cmap='nipy_spectral')
    plt.show()

In [ ]:
# Check if our prediction are FP or TP
for i in range(1,pred_nb+1):
    indices = np.where(all_pred==i+pred_nb)
    if(len(indices[0])==0):
        FP += 1
        np.putmask(pred,pred==i,125)
    else:
        gt_ind = gt[indices[0][0]][indices[0][1]]-1
        pred_ind = i -1
        # Get the coordinate of the intersection
        xmin = indices[0].min()
        xmax = indices[0].max()
        ymin = indices[1].min()
        ymax = indices[1].max()
        # Overlap calculation
        SI = (xmax - xmin)*(ymax- ymin )
        SA = (bb_pred[pred_ind][1]-bb_pred[pred_ind][0])*(bb_pred[pred_ind][3]-bb_pred[pred_ind][2])
        SB = (bb_gt[gt_ind][1]-bb_gt[gt_ind][0])*(bb_gt[gt_ind][3]-bb_gt[gt_ind][2])
        SU = SA + SB - SI
        overlap = SI / SU
        # If the overlap rate is above 20% we consider it as a TP
        if(overlap>0.2):
            TP += 1
            np.putmask(pred,pred==i,255)
        else:
            FP += 1
            np.putmask(pred,pred==i,125)

In [ ]:
# Display the result
if(display):
    plt.figure(figsize=(50,50))
    plt.subplot(131)
    plt.imshow(pred, cmap='nipy_spectral')
    plt.subplot(132)
    plt.imshow(gt, cmap='nipy_spectral')
    plt.show()

In [ ]:
# Finding the FN
for i in range(1,gt_nb+1):
    indices = np.where(all_gt==i+gt_nb)
    if(len(indices[0])==0):
        FN += 1
        np.putmask(gt,gt==i,180)
np.putmask(gt,gt!=180,0)

In [ ]:
# Putting the TP on the green channel and the FP on the red channel
maskArray[:,:,1]=maskArray[:,:,0]
np.putmask(maskArray[:,:,1],maskArray[:,:,1]!=255,0)
np.putmask(maskArray[:,:,0],maskArray[:,:,0]==255,0)
np.putmask(maskArray[:,:,0],maskArray[:,:,0]!=0,255)

In [ ]:
# Display the result
if(display):
    plt.figure(figsize=(50,50))
    plt.subplot(131)
    plt.imshow(maskArray)
    plt.show()

In [ ]:
# Opening the original image
im = op.OpenSlide(filename)
imload = im.read_region((0,0), 1, im.level_dimensions[1])

In [ ]:
# Adding transparency on the mask
maskArray[:,:,3]=0
maskArray[:,:,3] += maskArray[:,:,0] + maskArray[:,:,1] + maskArray[:,:,2]
np.putmask(maskArray[:,:,3],maskArray[:,:,3]!=0,transparency)

In [ ]:
# Merge the mask and image and save it
maskImage = Image.fromarray(maskArray, 'RGBA')
Image.alpha_composite(imload, maskImage).save(output_image)

In [ ]:
# Calculate the Fscore
Fscore = (2*TP)/(2*TP+FP+FN)
print("Fscore : ", Fscore)